In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 
import os
from tqdm import tqdm
import json
import time
import random
import pickle

In [2]:
df=pd.read_csv(r'E:\capstone\code\WLASL-master\WLASL-master\start_kit\data.csv',dtype={'video_id': str})
embedding_data=pd.read_csv(r'E:\capstone\code\WLASL-master\WLASL-master\start_kit\embedding_data.csv')

In [3]:
df.head()

,gloss,video_id,xmin,ymin,xmax,ymax,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id
0,book,69241,385,37,885,720,25.0,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0
1,book,65225,190,25,489,370,25.0,-1,1,1,90,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0
2,book,68011,262,1,652,480,25.0,-1,1,2,110,valencia-asl,train,https://www.youtube.com/watch?v=0UsjUE-TXns,0
3,book,68208,123,19,516,358,25.0,60,1,3,113,lillybauer,train,https://www.youtube.com/watch?v=1QOYOZ3g-aY,0
4,book,68012,95,0,1180,720,25.0,-1,1,4,109,valencia-asl,train,https://www.youtube.com/watch?v=aGtIHKEdCds,0


In [4]:
embedding_data.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,gloss
0,-0.256340,0.374570,-0.004352,-0.031364,0.497100,0.500190,-0.316900,-0.029405,-0.223950,2.3810,...,-0.038286,-0.125790,0.34074,0.676660,0.034156,0.70225,-0.44008,0.159510,-0.401530,book
1,-0.063465,-0.160110,0.383710,-0.376820,-0.043688,0.167340,-0.358510,0.183800,-0.112990,2.0250,...,-0.317530,-0.162240,-0.36107,-0.247620,0.285030,-0.18506,-0.23737,0.916300,-0.054556,drink
2,-0.266570,-0.137170,0.235490,-0.267120,0.093324,0.175630,-0.333090,-0.817440,0.526550,1.5880,...,-0.167160,0.375520,0.57434,0.055414,-0.225450,0.30528,0.35608,-0.464960,0.139140,computer
3,0.417900,0.034876,-0.187950,-0.135520,0.135230,-0.173070,-0.101180,0.000775,0.099486,3.1386,...,0.179180,0.087482,-0.12456,-0.098174,-0.221110,-0.13060,0.26873,0.064549,-0.024694,before
4,-0.020407,-0.495840,0.024120,0.102360,0.199390,0.064525,-0.095029,-0.604110,-0.059009,2.2198,...,-0.605170,0.940230,-0.55998,-0.169490,0.068575,0.48710,0.37698,-0.420090,-0.031210,chair


In [77]:
np.unique(df.gloss[:303]).shape

(11,)

In [78]:
temp_df=df[:303]

In [9]:
data={}
for i in tqdm(range(0,17000,500)):
    v_data = pickle.load(open(r'E:\capstone\code\WLASL-master\WLASL-master\start_kit\v_data_bw_{}.pkl'.format(i), "rb"))
    for j in v_data.keys():
        if j in list(temp_df['video_id']) :
            data[j]=v_data[j]

100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [01:49<00:00,  3.22s/it]


In [10]:
train_df=temp_df[temp_df['split']=="train"]
test_df=temp_df[temp_df['split']=="test"]

In [11]:
train_df.head()

,gloss,video_id,xmin,ymin,xmax,ymax,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id
0,book,69241,385,37,885,720,25.0,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0
1,book,65225,190,25,489,370,25.0,-1,1,1,90,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0
2,book,68011,262,1,652,480,25.0,-1,1,2,110,valencia-asl,train,https://www.youtube.com/watch?v=0UsjUE-TXns,0
3,book,68208,123,19,516,358,25.0,60,1,3,113,lillybauer,train,https://www.youtube.com/watch?v=1QOYOZ3g-aY,0
4,book,68012,95,0,1180,720,25.0,-1,1,4,109,valencia-asl,train,https://www.youtube.com/watch?v=aGtIHKEdCds,0


In [44]:
data['05724'].shape

(47, 320, 240)

In [162]:
training_input=[]
training_output=[]
testing_output=[]
testing_input=[]
training_word=[]
testing_word=[]

for i in tqdm(range(train_df.shape[0])):
    if train_df.iloc[i]['gloss'] in list(embedding_data.gloss) and train_df.iloc[i]['video_id'] in data.keys() :
        if(data[train_df.iloc[i]['video_id']].shape[0]>=40):
            training_input.append(data[train_df.iloc[i]['video_id']][:40,:,:])
            training_output.append(embedding_data[embedding_data['gloss']==train_df.iloc[i]['gloss']].values)
            training_word.append(train_df.iloc[i]['gloss'])

for i in tqdm(range(test_df['video_id'].shape[0])):
    if test_df.iloc[i]['gloss'] in list(embedding_data.gloss) and test_df.iloc[i]['video_id'] in data.keys():
        if(data[test_df.iloc[i]['video_id']].shape[0]>=40):
            testing_input.append(data[test_df.iloc[i]['video_id']][:40,:,:])
            testing_output.append(embedding_data[embedding_data['gloss']==test_df.iloc[i]['gloss']].values)
            testing_word.append(test_df.iloc[i]['gloss'])
    
training_input=np.asarray(training_input)
testing_input=np.asarray(testing_input)
training_output=np.asarray(training_output)
testing_output=np.asarray(testing_output)

100%|█████████████████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 487.20it/s]


In [163]:
training_input.shape,testing_input.shape,training_output.shape,testing_output.shape

((100, 40, 320, 240), (25, 40, 320, 240), (100, 1, 301), (25, 1, 301))

In [157]:
training_input[0].shape

(40, 320, 240)

In [167]:
training_output=training_output.reshape((100,301))
testing_output=testing_output.reshape((25,301))

In [174]:
training_output=training_output[:,:-1]
testing_output=testing_output[:,:-1]


In [175]:
np.unique(training_word).shape,np.unique(testing_word).shape

((11,), (10,))

In [176]:
training_input_3d=[]
for i in tqdm(training_input):
    data_3d=[]
    for j in i:
        temp_data=[]
        for x in range(0,3):
            temp_data.append(j)
        temp_data=np.asarray(temp_data)
        temp_data=temp_data.reshape((320,240,3))
        data_3d.append(temp_data)
    data_3d=np.asarray(data_3d)
    
    print(data_3d.shape)
            
    
    training_input_3d.append(data_3d) 
    

 14%|███████████▏                                                                    | 14/100 [00:00<00:00, 134.60it/s]

(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)


 43%|██████████████████████████████████▍                                             | 43/100 [00:00<00:00, 134.20it/s]

(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)


 67%|█████████████████████████████████████████████████████▌                          | 67/100 [00:00<00:00, 123.02it/s]

(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)

 90%|████████████████████████████████████████████████████████████████████████        | 90/100 [00:00<00:00, 116.57it/s]


(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 119.47it/s]


(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)
(40, 320, 240, 3)


In [177]:
training_input_3d=np.asarray(training_input_3d)

In [178]:
training_input_3d.shape

(100, 40, 320, 240, 3)

In [179]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed

video = Input(shape=(40,320,240,3))
cnn_base = VGG16(input_shape=(320,240,3),
                 weights="imagenet",
                 include_top=False
                )
cnn_out = GlobalAveragePooling2D()(cnn_base.output)
cnn = Model(inputs=cnn_base.input, outputs=cnn_out)
cnn.trainable = False
encoded_frames = TimeDistributed(cnn)(video)
encoded_sequence = LSTM(512)(encoded_frames)
hidden_layer = Dense(512, activation="relu")(encoded_sequence)
outputs = Dense(300, activation="relu")(hidden_layer)
model = Model([video], outputs)

model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=["categorical_accuracy"]) 

In [183]:
training_input_3d.shape

(100, 40, 320, 240, 3)

In [184]:
x_train=np.asarray(training_input_3d).astype(np.int)

y_train=np.asarray(training_output).astype(np.float)

In [185]:
model.fit(x_train, y_train, batch_size =32, epochs=1)


ResourceExhaustedError:  OOM when allocating tensor with shape[1280,320,240,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node functional_9/time_distributed_13/functional_7/block1_conv1/Relu (defined at <ipython-input-185-dbebe3f2e90f>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_16664]

Function call stack:
train_function


In [130]:
import tensorflow as tf
from keras.applications import VGG16
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Flatten, Input
import keras 

frames = Input(shape=(40, 320, 240, 3))
base_in = Input(shape=(320,240,3))

    
base_model = VGG16(weights='imagenet',include_top=False,input_shape=(320,240,3))
x = Flatten()(base_model.output)
x = TimeDistributed(Flatten())(x)
x = LSTM(units = 256, return_sequences=False, dropout=0.2)(x)
x = Dense(301, activation='softmax')(x)



ValueError: Please initialize `TimeDistributed` layer with a `tf.keras.layers.Layer` instance. You passed: Tensor("flatten_2/Reshape:0", shape=(None, 35840), dtype=float32)

In [126]:
import tensorflow as tf
from keras.applications import VGG16
from keras.layers import LSTM, Dense, TimeDistributed
import keras 

inputs = tf.keras.Input(shape=(40, 320, 240, 3))

vgg=VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="relu",
)
outputs = tf.keras.layers.TimeDistributed(vgg)(inputs)

model = tf.keras.layers.LSTM(100,activation='relu')(outputs)
model = tf.keras.layers.LSTM(100,activation='relu')(model)
model = tf.keras.layers.Dense(100,activation='relu')(outputs)




ValueError: Input 0 of layer lstm_13 is incompatible with the layer: expected ndim=3, found ndim=5. Full shape received: [None, 40, 10, 7, 512]

In [119]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])


In [121]:
model.fit(training_input_3d, training_output, batch_size =32, epochs=1)


ValueError: in user code:

    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
        outputs = m

In [35]:
training_input_3d.shape

(134,)

In [36]:
training_output.shape

(134, 301)